In [86]:
import pandas as pd
import numpy as np

In [87]:
import warnings
warnings.filterwarnings(action='ignore')

### 전처리된 데이터 불러오기

In [88]:
baby_growth_df = pd.read_csv("./baby_growth_df.csv")
baby_growth_df

,days,height,weight,gender
0,0,52.0,3.78,1
1,0,52.0,3.78,1
2,0,50.0,3.20,1
3,1,51.0,2.45,1
4,1,50.0,2.80,1
...,...,...,...,...
79344,365,74.6,9.60,0
79345,365,74.6,8.60,0
79346,365,74.6,9.40,0
79347,365,74.6,9.40,0


In [89]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

### height 예측 모델링

In [90]:
# 독립 변수와 종속 변수 분리
X = baby_growth_df[['days','gender']]
y = baby_growth_df['height']

# train/test 데이터 분리, 성별 비중 유지
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=X['gender'], random_state=42)

### (1) LinearRegression - height 예측 모델

In [91]:
# linear regression 모델 학습
lr_height = LinearRegression()
lr_height.fit(X_train, y_train)

# test 데이터로 예측
y_pred = lr_height.predict(X_test)

# 모델 성능 평가
r2 = r2_score(y_test, y_pred)
print('R-squared:', r2)

# MSE 평가
mse = mean_squared_error(y_test, y_pred)
print("LinearRegression MSE:", mse)

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(lr_height.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(lr_height.score(X_test, y_test)))

R-squared: 0.9053437089619766
LinearRegression MSE: 3.52819616917368
훈련 세트 정확도: 0.905
테스트 세트 정확도: 0.905


### (2) RandomForestRegressor - height 예측 모델

In [92]:
# RandomForestRegressor 모델 객체 생성
rf_height = RandomForestRegressor(random_state=42)

# 모델 학습
rf_height.fit(X_train, y_train)

# 모델 평가 - 예측값 생성
y_pred = rf_height.predict(X_test)

# 모델 평가 - 평균 제곱 오차 계산
mse = mean_squared_error(y_test, y_pred)
print("RandomForestRegressor Mean Squared Error:", mse)

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(rf_height.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(rf_height.score(X_test, y_test)))

RandomForestRegressor Mean Squared Error: 0.25375292488684353
훈련 세트 정확도: 0.993
테스트 세트 정확도: 0.993


### (3) XGBRegressor - height 예측 모델

In [93]:
# 모델 객체 생성
xgb_height = XGBRegressor(objective='reg:squarederror', tree_method='hist', random_state=42)

# 모델 학습
xgb_height.fit(X_train, y_train)

# 검증용 데이터를 사용하여 예측 수행
y_pred = xgb_height.predict(X_test)

# 모델 성능 평가
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"xgboost MSE: {mse:.3f}")
print(f"xgboost MAE: {mae:.3f}")
print(f"xgboost R-squared: {r2:.3f}")

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(xgb_height.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(xgb_height.score(X_test, y_test)))

xgboost MSE: 0.316
xgboost MAE: 0.204
xgboost R-squared: 0.992
훈련 세트 정확도: 0.992
테스트 세트 정확도: 0.992


### RandomForestRegressor hyperparameter tuning

In [9]:
# 하이퍼파라미터 후보 값들
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# GridSearchCV 수행
rf_height = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(rf_height, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터와 그 때의 성능 출력
print("Best hyperparameters:", grid_search.best_params_)
print("Best CV score:", -grid_search.best_score_)

# 최적의 하이퍼파라미터로 모델 학습 및 평가
rf_height_best = RandomForestRegressor(n_estimators=grid_search.best_params_['n_estimators'],
                                 max_depth=grid_search.best_params_['max_depth'],
                                 min_samples_split=grid_search.best_params_['min_samples_split'],
                                 random_state=42)
rf_height_best.fit(X_train, y_train)
y_pred_rf = rf_height_best.predict(X_test)
print("Test MSE:", mean_squared_error(y_test, y_pred_rf))

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(rf_height_best.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(rf_height_best.score(X_test, y_test)))


Best hyperparameters: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 50}
Best CV score: 0.2536826197356331
Test MSE: 0.2537769441848588
훈련 세트 정확도: 0.993
테스트 세트 정확도: 0.993


### (4) LightGBM - height 예측 모델

In [94]:
# LightGBM 모델링
lgbm_height = LGBMRegressor(random_state=42)
lgbm_height.fit(X_train, y_train)

# 예측값 생성
y_pred = lgbm_height.predict(X_test)

# MSE 평가
mse = mean_squared_error(y_test, y_pred)
print("lightGBM MSE:", mse)

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(lgbm_height.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(lgbm_height.score(X_test, y_test)))

lightGBM MSE: 0.30606518685357387
훈련 세트 정확도: 0.992
테스트 세트 정확도: 0.992


### (5) GradientBoosting - height 예측 모델

In [95]:
# Gradient Boosting 모델 학습
gbr_height = GradientBoostingRegressor(random_state=42)
gbr_height.fit(X_train, y_train)

# 모델 예측
y_pred = gbr_height.predict(X_test)

# 모델 평가
mse = mean_squared_error(y_test, y_pred)
print("GradientBoostingRegressor MSE:", mse)

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(gbr_height.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(gbr_height.score(X_test, y_test)))

GradientBoostingRegressor MSE: 0.35741078812332194
훈련 세트 정확도: 0.990
테스트 세트 정확도: 0.990


### Weight 예측 모델링

In [96]:
# 독립 변수와 종속 변수 분리
X_data = baby_growth_df[['days','gender','height']]
y_target = baby_growth_df['weight']

# train/test 데이터 분리, 성별 비중 유지
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.25, stratify=X_data['gender'], random_state=42)

### (1) RandomForestRegressor - weight 예측 모델

In [47]:
# 하이퍼파라미터 후보 값들
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# GridSearchCV 수행
rf_weight = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(rf_weight, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터와 그 때의 성능 출력
print("Best hyperparameters:", grid_search.best_params_)
print("Best CV score:", -grid_search.best_score_)

# 최적의 하이퍼파라미터로 모델 학습 및 평가
rf_weight_best = RandomForestRegressor(n_estimators=grid_search.best_params_['n_estimators'],
                                 max_depth=grid_search.best_params_['max_depth'],
                                 min_samples_split=grid_search.best_params_['min_samples_split'],
                                 random_state=42)
rf_weight_best.fit(X_train, y_train)
y_pred_rf = rf_weight_best.predict(X_test)
print("Test MSE:", mean_squared_error(y_test, y_pred_rf))

# 정확도 평가
print("훈련 세트 정확도: {:.3f}".format(rf_weight_best.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(rf_weight_best.score(X_test, y_test)))


Best hyperparameters: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 100}
Best CV score: 0.3465750289982933
Test MSE: 0.34468287750259985
훈련 세트 정확도: 0.894
테스트 세트 정확도: 0.891


AttributeError: 'RandomForestRegressor' object has no attribute 'oob_score_'

### (2) LinearRegressor - weight 예측 모델

In [97]:
lr_weight = LinearRegression()

lr_weight.fit(X_train, y_train)
lr_weight.score(X_test, y_test)

print("훈련 세트 정확도: {:.3f}".format(lr_weight.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(lr_weight.score(X_test, y_test)))

훈련 세트 정확도: 0.885
테스트 세트 정확도: 0.885


### (3) GradientBoostingRegressor - weight 예측 모델

In [98]:
gbr_weight = GradientBoostingRegressor()

gbr_weight.fit(X_train, y_train)

# 평가
print("훈련 세트 정확도: {:.3f}".format(gbr_weight.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(gbr_weight.score(X_test, y_test)))

훈련 세트 정확도: 0.892
테스트 세트 정확도: 0.891


In [115]:
# 사용자로부터 데이터 입력받기
input_days = int(input("생후 일 수를 입력하세요: "))
input_height = float(input("아기의 현재 키를 입력하세요 (단위: cm): "))
input_weight = float(input("아기의 현재 몸무게를 입력하세요 (단위: kg): "))
input_gender = input("아기의 성별을 입력하세요 (F/M): ")

# 입력받은 데이터로 1개월, 3개월, 6개월 뒤의 키 예측
if input_gender == 'F':
    gender = 0
elif input_gender == 'M':
    gender = 1
else:
    print("성별은 'F' 또는 'M' 중에서 입력해주세요.")
    exit()

one_month_pred_height = lr_height.predict([[input_days + 30, gender]])
three_month_pred_height = lr_height.predict([[input_days + 90, gender]]) 
six_month_pred_height = lr_height.predict([[input_days + 180, gender]]) 

one_month_pred_weight = lr_weight.predict([[input_days + 30, gender, one_month_pred_height]])
three_month_pred_weight = lr_weight.predict([[input_days + 90, gender, three_month_pred_height]]) 
six_month_pred_weight = lr_weight.predict([[input_days + 180, gender, six_month_pred_height]]) 

# 결과 출력
print(f"아기의 1개월 뒤 예상 키는 {one_month_pred_height[0]:.1f}cm 입니다.")
print(f"아기의 3개월 뒤 예상 키는 {three_month_pred_height[0]:.1f}cm 입니다.")
print(f"아기의 6개월 뒤 예상 키는 {six_month_pred_height[0]:.1f}cm 입니다.")
print(f"아기의 1개월 뒤 예상 몸무게는 {one_month_pred_weight[0]:.1f}kg 입니다.")
print(f"아기의 3개월 뒤 예상 몸무게는 {three_month_pred_weight[0]:.1f}kg 입니다.")
print(f"아기의 6개월 뒤 예상 몸무게는 {six_month_pred_weight[0]:.1f}kg 입니다.")

아기의 1개월 뒤 예상 키는 68.5cm 입니다.
아기의 3개월 뒤 예상 키는 73.4cm 입니다.
아기의 6개월 뒤 예상 키는 80.9cm 입니다.
아기의 1개월 뒤 예상 몸무게는 8.3kg 입니다.
아기의 3개월 뒤 예상 몸무게는 9.7kg 입니다.
아기의 6개월 뒤 예상 몸무게는 11.7kg 입니다.


In [116]:
# for height graph
graph_height = [input_height] # 0일 뒤 키 == 현재 입력 키
x_height_days = list(range(1,181)) # 1일 뒤부터 180일 뒤 까지
for i in range(1,181):
    graph_height.append(lr_height.predict([[input_days + i, gender]])[0])
graph_height


[65.0,
 66.06768014924604,
 66.15041420682392,
 66.23314826440182,
 66.3158823219797,
 66.3986163795576,
 66.48135043713549,
 66.56408449471337,
 66.64681855229126,
 66.72955260986916,
 66.81228666744704,
 66.89502072502492,
 66.97775478260282,
 67.06048884018071,
 67.14322289775859,
 67.22595695533649,
 67.30869101291438,
 67.39142507049226,
 67.47415912807016,
 67.55689318564804,
 67.63962724322593,
 67.72236130080381,
 67.8050953583817,
 67.8878294159596,
 67.97056347353748,
 68.05329753111538,
 68.13603158869327,
 68.21876564627115,
 68.30149970384905,
 68.38423376142693,
 68.46696781900482,
 68.5497018765827,
 68.6324359341606,
 68.71516999173849,
 68.79790404931637,
 68.88063810689427,
 68.96337216447216,
 69.04610622205004,
 69.12884027962792,
 69.21157433720582,
 69.29430839478371,
 69.3770424523616,
 69.45977650993949,
 69.54251056751738,
 69.62524462509526,
 69.70797868267316,
 69.79071274025104,
 69.87344679782893,
 69.95618085540681,
 70.03891491298471,
 70.1216489705626,
 

In [117]:
# for weight graph
graph_weight = [input_weight] # 0일 뒤 몸무게 == 현재 입력 몸무게
x_weight_days = list(range(1,181)) # 1일 뒤부터 180일 뒤 까지
for i in range(1,181):
    graph_weight.append(lr_weight.predict([[input_days + i, gender, graph_height[i]]])[0])
graph_weight


[8.0,
 7.658965861123754,
 7.681471742213439,
 7.703977623303128,
 7.72648350439281,
 7.748989385482499,
 7.771495266572188,
 7.79400114766187,
 7.816507028751559,
 7.839012909841244,
 7.86151879093093,
 7.884024672020615,
 7.906530553110301,
 7.929036434199993,
 7.951542315289679,
 7.974048196379364,
 7.996554077469053,
 8.019059958558735,
 8.041565839648424,
 8.06407172073811,
 8.086577601827795,
 8.10908348291748,
 8.13158936400717,
 8.154095245096858,
 8.176601126186544,
 8.199107007276229,
 8.221612888365918,
 8.244118769455604,
 8.266624650545289,
 8.289130531634974,
 8.311636412724663,
 8.334142293814345,
 8.356648174904034,
 8.37915405599372,
 8.401659937083405,
 8.424165818173098,
 8.446671699262783,
 8.469177580352468,
 8.491683461442154,
 8.51418934253184,
 8.536695223621528,
 8.55920110471121,
 8.5817069858009,
 8.604212866890588,
 8.62671874798027,
 8.649224629069963,
 8.671730510159648,
 8.694236391249333,
 8.716742272339019,
 8.739248153428708,
 8.761754034518393,
 8.784